This script gets a map representing the main effect in the MCSE study, extracts the peak coordinates from it
and then fetches anatomical labels for these regions


In [1]:
# Let us load the image obtained from the previous group analysis.
# we are interested in the low-high salience contrast only
import os
from nilearn.image import load_img
write_dir = 'results'
key = 'low-high salience'
filename = os.path.join(write_dir, f'group_{key}_z_score.nii.gz')
z_map = load_img(filename)

In [2]:
from nilearn.reporting import get_clusters_table
# we need to threshold the maps. We keep
threshold = 3.06

table = get_clusters_table(
    z_map, stat_threshold=threshold, cluster_threshold=20
)
table

,Cluster ID,X,Y,Z,Peak Stat,Cluster Size (mm3)
0,1,-33.0,-94.0,17.0,8.106788,1080
1,1a,-27.0,-85.0,23.0,4.516470,
2,2,-6.0,-76.0,-25.0,7.658465,1350
3,2a,0.0,-76.0,-31.0,5.051207,
4,2b,9.0,-73.0,-19.0,4.105934,
5,2c,6.0,-76.0,-28.0,3.863424,
6,3,48.0,11.0,38.0,7.639234,3429
7,3a,36.0,11.0,32.0,5.088789,
8,3b,51.0,2.0,50.0,4.103084,
9,4,-30.0,-61.0,53.0,7.382171,1755


We see 37 main peaks organized in 14 clusters.
Now let us try to make sense of these clusters by observing then in the Julich atlas space.

To fetch the Julich-Brain from EBRAINS we rely on the `siibra` library. So we first need to import it (or install and then import if not already installed)

In [4]:
try:
    import siibra
except:
    !pip install siibra
    import siibra
print(siibra.__version__)
assert siibra.__version__ >= "0.4a33"

[autoreload of scipy.sparse.linalg._isolve.iterative failed: Traceback (most recent call last):
  File "/home/yasmin/miniconda3/envs/ebrains-hands-on/lib/python3.9/site-packages/scipy/__init__.py", line 214, in __getattr__
    return globals()[name]
KeyError: '_lib'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yasmin/miniconda3/envs/ebrains-hands-on/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/home/yasmin/miniconda3/envs/ebrains-hands-on/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/home/yasmin/miniconda3/envs/ebrains-hands-on/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_modu

0.4a35


Now we can fetch the Julich-Brain parcellation 

In [ ]:
julich_brain = siibra.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_9
print(julich_brain.description)

# the original maximum probability map object with split hemispheres
julich_mpm = julich_brain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.LABELLED
)

# single-volume compressed version
julich_mpm_compressed = julich_mpm.compress()
parcellation_map_niimg = julich_mpm_compressed.fetch()